In [7]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py


--2024-06-30 15:45:06--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-30 15:45:06 (31.4 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [8]:
import minsearch

In [9]:
import json


In [10]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
from elasticsearch import Elasticsearch


In [12]:
es_clent = Elasticsearch('http://localhost:9200')

In [13]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name ="course-qiestions"
es_clent.indices.create(index=index_name, body=index_setting)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-qiestions/FVfKVglXTkS7vPwCZZxZaA] already exists')

In [ ]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for doc in tqdm(documents):
    es_clent.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:31<00:00, 30.29it/s]


In [ ]:
query = 'I just discovered the course, can i still join it?'

In [ ]:
query

'How do I execute a command in a running docker container?'

In [ ]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es_clent.search(index=index_name, body=search_query)

In [ ]:
result_doc=[]
for hit in response['hits']['hits']:
    result_doc.append(hit['_source'])

In [ ]:
#### question 3

In [ ]:
q = 'How do I execute a command in a running docker container?'

In [ ]:
search_q = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

In [ ]:
response = es_clent.search(index=index_name, body=search_q)

In [ ]:
result_doc=[]
for hit in response['hits']['hits']:
    result_doc.append(hit['_score'])

In [ ]:
result_doc

[84.17781, 84.17781, 75.66571, 75.66571, 72.20392]

In [ ]:
#### question 4

In [ ]:
search_query_ml = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": q,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
              "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [ ]:
response = es_clent.search(index=index_name, body=search_query_ml)

In [ ]:
result_doc=[]
for hit in response['hits']['hits']:
    result_doc.append(hit['_source'])

In [ ]:
result_doc[3]['question']

'How do I copy files from my local machine to docker container?'

In [ ]:
#### question 5

In [ ]:
from openai import OpenAI

In [ ]:
client = OpenAI()

In [ ]:
prompt_templete = """
You are a course teaching assistant. Answer the QUESTION based on CONTEXT based on the CONTEXT from the FAQ database.
Use only the fact from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}

""".strip()

In [ ]:
context = ''

for doc in result_doc:
    context = context + f"section:{doc['section']}\nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    

In [ ]:
promt = prompt_templete.format(question = q, context= context).strip()

In [ ]:
print(promt)

You are a course teaching assistant. Answer the QUESTION based on CONTEXT based on the CONTEXT from the FAQ database.
Use only the fact from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: How do I execute a command in a running docker container?

CONTEXT: section:5. Deploying Machine Learning Models
question:How do I debug a docker container?
answer:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

section:5. Deploying Machine Learning Models
question:How do I debug a docker container?
answer:Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is 

In [ ]:
len(promt)


2606

In [ ]:
#### question 6

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o', 
    messages=[{"role":"user", "content":promt}]
)

In [ ]:
response

ChatCompletion(id='chatcmpl-9fqnp9ZrWVS1qcwUCK4yAAFMhS1SA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To execute a command in a running Docker container, follow these steps:\n\n1. Use `docker ps` to find the container ID of the running container.\n2. Use `docker exec -it <container-id> <command>` to execute the command in the specified container.\n\nFor example, to start a bash session in the container, you would run:\n```bash\ndocker exec -it <container-id> bash\n```\n\n(Source: Marcos MJD)', role='assistant', function_call=None, tool_calls=None))], created=1719761449, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_ce0793330f', usage=CompletionUsage(completion_tokens=92, prompt_tokens=575, total_tokens=667))

In [ ]:
response.choices[0].message.content

'To execute a command in a running Docker container, follow these steps:\n\n1. Use `docker ps` to find the container ID of the running container.\n2. Use `docker exec -it <container-id> <command>` to execute the command in the specified container.\n\nFor example, to start a bash session in the container, you would run:\n```bash\ndocker exec -it <container-id> bash\n```\n\n(Source: Marcos MJD)'

In [ ]:
import tiktoken

In [ ]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [ ]:
encoded_prompt = encoding.encode(promt)

In [ ]:
num_tokens = len(encoded_prompt)
print(f"Number of tokens: {num_tokens}")

Number of tokens: 568
